In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MeanShift
import numpy as np

In [26]:
logs = pd.read_csv("data\merged_logs_small.csv")
df = logs[["Log"]].iloc[0:500]
df
#logs

,Log
0,081109 203615 148 INFO dfs.DataNode$PacketResp...
1,081109 203807 222 INFO dfs.DataNode$PacketResp...
2,081109 204005 35 INFO dfs.FSNamesystem: BLOCK*...
3,081109 204015 308 INFO dfs.DataNode$PacketResp...
4,081109 204106 329 INFO dfs.DataNode$PacketResp...
...,...
495,081110 103819 19 INFO dfs.FSDataset: Deleting ...
496,081110 103821 19 INFO dfs.FSDataset: Deleting ...
497,081110 103827 26 INFO dfs.FSNamesystem: BLOCK*...
498,081110 103840 19 INFO dfs.FSDataset: Deleting ...


In [29]:
# TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['Log'])

# mean shift
mean_shift = MeanShift()
mean_shift.fit(tfidf_matrix.toarray())
labels = mean_shift.labels_

# adding cluster labels
df['cluster'] = labels

# slecting representative samples
def get_representative_logs(df, tfidf_matrix, mean_shift, num_representative=3):
    representative_logs = []
    for cluster in np.unique(labels):
        cluster_points = df[df['cluster'] == cluster]
        cluster_center = mean_shift.cluster_centers_[cluster]
        # distance of log to its cluster center
        distances = np.linalg.norm(tfidf_matrix[cluster_points.index].toarray() - cluster_center, axis=1)
        # indicies of closest logs (most representative of cluster)
        closest_indices = np.argsort(distances)
        step = max(1, len(closest_indices) // num_representative)
        selected_indices = closest_indices[::step][:num_representative]
        # append most representative logs for each cluster
        for idx in selected_indices:
            representative_logs.append(cluster_points.iloc[idx]['Log'])
    return representative_logs

# SET number of desired examples
num_representative = 3

# call func
representative_logs = get_representative_logs(df, tfidf_matrix, mean_shift, num_representative)

print("Representative Logs:")
for log in representative_logs:
    print(log)

Representative Logs:
081110 103111 30 INFO dfs.FSNamesystem: BLOCK* NameSystem.delete: blk_-8403618945096094999 is added to invalidSet of 10.251.42.9:50010
081110 011133 5157 INFO dfs.DataNode$PacketResponder: Received block blk_-7007208336332784130 of size 67108864 from /10.251.27.63
081109 211918 1777 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_4886940526690879848 terminating
